In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
from datetime import datetime, timezone
import time

def load_parquet_pandas(path, batch_size=50):
    start = time.time()
    parquet_file = pq.ParquetFile(path)
    df_chunk = next(parquet_file.iter_batches(batch_size=batch_size)).to_pandas()
    print(f"Время чтения: {time.time() - start:.2f} секунд")
    return df_chunk

In [2]:
def normalize_watchid(watch_id):
    """Приводит watchID к единому формату - строковому представлению целого числа"""
    if isinstance(watch_id, (int, np.integer)):
        return str(watch_id)
    elif isinstance(watch_id, float):
        # Преобразуем float через int для точности
        return str(int(watch_id))
    elif isinstance(watch_id, str):
        # Обрабатываем экспоненциальную запись
        if 'e+' in watch_id.lower() or 'e-' in watch_id.lower():
            try:
                # Преобразуем научную нотацию в целое число
                return str(int(float(watch_id)))
            except (ValueError, OverflowError):
                return watch_id
        else:
            # Убираем лишние символы для обычных чисел
            return watch_id.strip().replace("'", "").replace('"', '')
    else:
        return str(watch_id)

In [3]:
def read_matching_hits_normalized(hits_path, visits_watchids, batch_size=10000):
    """Читает hits с нормализацией watchID"""
    parquet_file = pq.ParquetFile(hits_path)
    all_matching_hits = []
    
    # Нормализуем watchID из visits
    visits_watchids_normalized = [normalize_watchid(wid) for wid in visits_watchids]
    visits_watchids_set = visits_watchids_normalized
    
    print(f"Поиск {len(visits_watchids_set)} нормализованных watchID")
    
    for i, batch in enumerate(parquet_file.iter_batches(batch_size=batch_size)):
        df_chunk = batch.to_pandas()
        
        # Нормализуем watchID в hits
        df_chunk['ym:pv:watchID'] = df_chunk['ym:pv:watchID'].apply(normalize_watchid)
        
        # Фильтруем по нормализованным watchID
        matching_chunk = df_chunk[df_chunk['ym:pv:watchID'].isin(visits_watchids_set)]
        
        if len(matching_chunk) > 0:
            # Сохраняем оригинальную колонку для совместимости
            all_matching_hits.append(matching_chunk)
            print(f"Чанк {i+1}: найдено {len(matching_chunk)} совпадений")
            
            # Примеры найденных совпадений для проверки
            sample_matches = matching_chunk['ym:pv:watchID'].head(3).tolist()
            print(f"  Примеры найденных watchID: {sample_matches}")
        else:
            print(f"Чанк {i+1}: ничего не найдено!")
        if i == 10:
            break
    
    if all_matching_hits:
        result = pd.concat(all_matching_hits, ignore_index=True)
        print(f"✅ Всего найдено совпадений: {len(result)}")
        return result
    else:
        print("❌ Совпадений не найдено")
        return pd.DataFrame()

In [22]:
def explode_and_join(visits, hits):
    
    # Приводим типы и чистим данные
    visits['watchID'] = visits['watchID'].apply(normalize_watchid)
    hits['watchID'] = hits['watchID'].apply(normalize_watchid)
    visits['clientID'] = visits['clientID'].astype(str).str.strip()
    hits['clientID'] = hits['clientID'].astype(str).str.strip()
    
    # Диагностика после explode
    print(f"Размер visits после explode: {len(visits)}")
    print(f"Уникальных watchID в visits после explode: {visits['watchID'].nunique()}")
    
    # Проверяем пересечение watchID
    visits_watchids = set(visits['watchID'].unique())
    hits_watchids = set(hits['watchID'].unique())
    common_watchids = visits_watchids.intersection(hits_watchids)
    
    print(f"Общих watchID: {len(common_watchids)}")
    print(f"WatchID только в visits: {len(visits_watchids - hits_watchids)}")
    print(f"WatchID только в hits: {len(hits_watchids - visits_watchids)}")
    
    # Merge
    joined = visits.merge(
        hits, 
        how='left', 
        on=['watchID'], 
        suffixes=('_visit', '_hit')
    )
    
    # Диагностика после merge
    print(f"Размер после merge: {len(joined)}")
    print(f"Строк с URL (не NaN): {joined['URL'].notna().sum()}")
    print(f"Процент заполнения URL: {joined['URL'].notna().mean() * 100:.2f}%")
    
    # Проверяем примеры данных
    print("\nПримеры watchID из visits (первые 5):")
    print(visits['watchID'].head().tolist())
    print("Примеры watchID из hits (первые 5):")
    print(hits['watchID'].head().tolist())
    
    # Если URL все еще NaN, проверяем конкретные случаи
    if joined['URL'].notna().sum() == 0:
        print("\n⚠️ ВНИМАНИЕ: Все URL равны NaN!")
        print("Проверяем конкретные watchID:")
        sample_watchids = visits['watchID'].head(3).tolist()
        for wid in sample_watchids:
            in_hits = hits[hits['watchID'] == wid]
            print(f"watchID '{wid}': найдено в hits - {len(in_hits)} записей")
    
    # Обработка дат и сортировка
    joined['dateTime_visit'] = pd.to_datetime(joined['dateTime_visit'])
    joined['dateTime_hit'] = pd.to_datetime(joined['dateTime_hit'])
    joined = joined.sort_values(['visitID', 'dateTime_hit'])
    
    return joined

In [5]:
def filter_visits_by_hits(visits, hits_df):
    """
    Удаляет из visits строки с watchID, которых нет в hits
    """
    hits = hits_df.copy()
    
    # Получаем множество существующих watchID в hits
    existing_watchids = set(hits['watchID'].unique())
    print(f"Всего уникальных watchID в hits: {len(existing_watchids)}")
    
    # Фильтруем visits - оставляем только те watchID, которые есть в hits
    initial_count = len(visits)
    visits_filtered = visits[visits['watchID'].isin(existing_watchids)]
    filtered_count = len(visits_filtered)
    
    print(f"Отфильтровано visits: {initial_count} -> {filtered_count} строк")
    print(f"Удалено {initial_count - filtered_count} строк ({((initial_count - filtered_count)/initial_count)*100:.1f}%)")
    
    return visits_filtered.reset_index(drop=True)

In [6]:
visits_norm = load_parquet_pandas("data/2024_yandex_metrika_visits.parquet", 99999)
visits_norm.columns = visits_norm.columns.str.replace('ym:s:', '', regex=False)
visits = visits_norm.copy()

Время чтения: 0.99 секунд


In [7]:
# Парсим watchIDs
if isinstance(visits['watchIDs'].iloc[0], str):
    try:
        visits['watchIDs'] = visits['watchIDs'].apply(json.loads)
    except:
        try:
            visits['watchIDs'] = visits['watchIDs'].apply(ast.literal_eval)
        except:
            visits['watchIDs'] = visits['watchIDs'].str.strip("[]").str.replace("'", "").str.split(",")
# Explode
visits = visits.explode('watchIDs').rename(columns={'watchIDs': 'watchID'}).reset_index(drop=True)

# Приводим типы и чистим данные
visits['watchID'] = visits['watchID'].astype(str).str.strip()
visits['clientID'] = visits['clientID'].astype(str).str.strip()

In [8]:
s = set(visits['watchID'])

In [9]:
hits = read_matching_hits_normalized("data/2024_yandex_metrika_hits.parquet", s, 99999)

Поиск 1115902 нормализованных watchID
Чанк 1: найдено 194 совпадений
  Примеры найденных watchID: ['314716371219644544', '1378710678365274368', '1532321015153819904']
Чанк 2: найдено 184 совпадений
  Примеры найденных watchID: ['1843689614446428416', '1489513446411600128', '529329131393646976']
Чанк 3: найдено 277 совпадений
  Примеры найденных watchID: ['1542490554399719680', '612381436111749248', '612386899792756864']
Чанк 4: найдено 260 совпадений
  Примеры найденных watchID: ['173878631214612512', '62891990265364496', '63149368285069368']
Чанк 5: найдено 153 совпадений
  Примеры найденных watchID: ['465015336507015552', '465719789567345024', '740474594292400512']
Чанк 6: найдено 7 совпадений
  Примеры найденных watchID: ['967955283339116672', '156273831978205664', '63646493726736424']
Чанк 7: ничего не найдено!
Чанк 8: ничего не найдено!
Чанк 9: ничего не найдено!
Чанк 10: ничего не найдено!
Чанк 11: ничего не найдено!
✅ Всего найдено совпадений: 1075


In [10]:
hits.columns = hits.columns.str.replace('ym:pv:', '', regex=False)

In [11]:
hits.to_hdf('data/hits.h5', key='df', mode='w')

In [11]:
visits_filtered = filter_visits_by_hits(visits, hits)

Всего уникальных watchID в hits: 1075
Отфильтровано visits: 1116020 -> 1075 строк
Удалено 1114945 строк (99.9%)


In [13]:
visits_filtered.to_hdf('data/visits_f.h5', key='df', mode='w')

In [15]:
hits['URL']

0       https://priem.mai.ru/bachelor/programs/?city=М...
1       https://priem.mai.ru/bachelor/programs/item/pr...
2       https://priem.mai.ru/bachelor/programs/item/pr...
3                 https://priem.mai.ru/bachelor/programs/
4       https://priem.mai.ru/bachelor/programs/?city=М...
                              ...                        
1070    https://priem.mai.ru/orders/hostel/check-in202...
1071    https://priem.mai.ru/bachelor/tests/?referer=h...
1072    https://priem.mai.ru/bachelor/programs/item/sa...
1073             https://priem.mai.ru/orders/regulations/
1074    https://files.mai.ru/site/priem/documents/orde...
Name: URL, Length: 1075, dtype: object

In [23]:
jo = explode_and_join(visits_filtered, hits)

Размер visits после explode: 1075
Уникальных watchID в visits после explode: 1075
Общих watchID: 1075
WatchID только в visits: 0
WatchID только в hits: 0
Размер после merge: 1075
Строк с URL (не NaN): 1075
Процент заполнения URL: 100.00%

Примеры watchID из visits (первые 5):
['1649044085182562560', '357314722397159488', '380491393517748544', '166403020232589792', '166376451454271968']
Примеры watchID из hits (первые 5):
['314716371219644544', '1378710678365274368', '1532321015153819904', '83540290082963616', '83544208632971424']


In [26]:
jo['URL']

697    https://priem.mai.ru/master/deadlines/?referer...
700    https://priem.mai.ru/master/deadlines/?referer...
698                           https://priem.mai.ru/list/
699                    https://pre.mai.ru/events/public/
123                https://priem.mai.ru/master/programs/
                             ...                        
718    https://priem.mai.ru/bachelor/programs/?city=М...
642         https://priem.mai.ru/news/item.php?id=177110
643         https://priem.mai.ru/news/item.php?id=177110
379    https://priem.mai.ru/bachelor/programs/item/pr...
380             https://priem.mai.ru/orders/plan/common/
Name: URL, Length: 1075, dtype: object

In [15]:
jo.to_hdf('data/joins_f.h5', key='df', mode='w')

In [39]:
hits.head()

,watchID,pageViewID,counterID,clientID,counterUserIDHash,date,dateTime,title,pageCharset,goalsID,...,parsedParamsKey6,parsedParamsKey7,parsedParamsKey8,parsedParamsKey9,parsedParamsKey10,httpError,networkType,shareService,shareURL,shareTitle
0,314716371219644544,800211281,45231030,1.7069083296926144e+18,4228895924505989123,2024-02-03,2024-02-03 00:29:07,,utf-8,[],...,[],[],[],[],[],0,,NaN,NaN,NaN
1,1378710678365274368,839857608,45231030,1.7109681641272228e+18,914294369382034668,2024-03-20,2024-03-20 23:56:03,Проектирование и технология радиоэлектронных с...,utf-8,[],...,[],[],[],[],[],0,cellular,NaN,NaN,NaN
2,1532321015153819904,877459550,45231030,1.7115541247782973e+18,8356342311940082382,2024-03-27,2024-03-27 18:42:20,,utf-8,[],...,[],[],[],[],[],0,wi_fi,NaN,NaN,NaN
3,83540290082963616,796388768,45231030,1.6936839495997972e+18,2127754057760289469,2024-01-23,2024-01-23 19:31:19,Направления,utf-8,[],...,[],[],[],[],[],0,wi_fi,NaN,NaN,NaN
4,83544208632971424,796388768,45231030,1.6936839495997972e+18,2127754057760289469,2024-01-23,2024-01-23 19:31:35,,utf-8,[],...,[],[],[],[],[],0,wi_fi,NaN,NaN,NaN


In [ ]:
def page_bounce_rates(visits_df, key_pages):
    # visits_df: строки с одной сессией, колонки: startURL, pageViews, visitID, dateTime
    df = visits_df.copy()
    df['is_bounce'] = df['pageViews'] == 1
    # кто вошёл на ключевую страницу (startURL == key_page) — или выставим условие "visited page anywhere in session"
    rows = []
    for page in key_pages:
        entered = df[df['startURL'].str.contains(page, na=False)]
        bounce_rate = entered['is_bounce'].mean() if len(entered)>0 else np.nan
        rows.append({'page': page, 'sessions': len(entered), 'bounce_rate': bounce_rate})
    return pd.DataFrame(rows)


In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100000)

# Теперь можно смотреть больше данных
print(visits_filtered)

                 visitID  counterID              watchID        date             dateTime          dateTimeUTC  isNewUser                                           startURL                                             endURL  pageViews  visitDuration  bounce                 ipAddress regionCountry regionCity  regionCountryID  regionCityID             clientID     counterUserIDHash networkType goalsID goalsSerialNumber goalsDateTime goalsPrice goalsOrder  ... browserEngineVersion4 cookieEnabled javascriptEnabled screenFormat screenColors screenOrientation screenOrientationName  screenWidth screenHeight  physicalScreenWidth  physicalScreenHeight  windowClientWidth windowClientHeight parsedParamsKey1 parsedParamsKey2 parsedParamsKey3 parsedParamsKey4 parsedParamsKey5 parsedParamsKey6 parsedParamsKey7 parsedParamsKey8 parsedParamsKey9 parsedParamsKey10 lastsignRecommendationSystem lastsignMessenger
0     176155276604080158   45231030   176159369259647136  2022-01-27  2022-01-27 21:39:39  20

In [20]:
visits['startURL'].head()

0    https://priem.mai.ru/rating/?referer=https:%2F...
1    https://priem.mai.ru/rating/?referer=https:%2F...
2    https://priem.mai.ru/rating/?referer=https:%2F...
3    https://priem.mai.ru/rating/?referer=https:%2F...
4    https://priem.mai.ru/rating/?referer=https:%2F...
Name: startURL, dtype: object

In [21]:
page_bounce_rates(visits, ['https://priem.mai.ru'])

,page,sessions,bounce_rate
0,https://priem.mai.ru,1417477,0.162434


In [22]:
jo['URL']

203    https://priem.mai.ru/master/programs/item/inde...
205    https://priem.mai.ru/master/programs/item/inde...
204    https://priem.mai.ru/upload/iblock/779/24.04.0...
662                                https://priem.mai.ru/
663                                https://priem.mai.ru/
                             ...                        
80                                 https://priem.mai.ru/
435        https://priem.mai.ru/bachelor/tests/#internal
478    https://priem.mai.ru/orders/testing/magistracy...
479                       https://priem.mai.ru/calendar/
480                     https://lk.mai.ru/accounts/login
Name: URL, Length: 887, dtype: object

### Блуждание по сайту (Wandering / aimless navigation)

Признаки: много переходов

In [42]:
jo

,visitID,counterID_visit,watchID,date_visit,dateTime_visit,dateTimeUTC,isNewUser,startURL,endURL,pageViews,...,parsedParamsKey6_hit,parsedParamsKey7_hit,parsedParamsKey8_hit,parsedParamsKey9_hit,parsedParamsKey10_hit,httpError,networkType_hit,shareService,shareURL,shareTitle
440,NaN,NaN,330465190609086,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,cellular,NaN,NaN,NaN
441,NaN,NaN,334977274478870,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,cellular,NaN,NaN,NaN
442,NaN,NaN,344909032980670,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,cellular,NaN,NaN,NaN
272,NaN,NaN,360942905065484,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,,NaN,NaN,NaN
273,NaN,NaN,364849675894827,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,NaN,NaN,2051949958684737792,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,,NaN,NaN,NaN
428,NaN,NaN,2070313016432001280,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,,NaN,NaN,NaN
429,NaN,NaN,2070318082777415936,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,,NaN,NaN,NaN
799,NaN,NaN,2074717348051812608,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,[],[],[],[],[],0,,NaN,NaN,NaN


In [40]:
def detect_wandering(joined_df, pageview_threshold=8):
    # joined_df: exploded hits joined to visits, must contain visitID, title, URL, dateTime_hit
    sessions = joined_df.groupby('visitID').agg({
        'URL':'nunique',
        'watchID':'count',
        'dateTime_visit':'min'
    }).rename(columns={'watchID':'hits','URL':'unique_pages'})
    # wandering — много переходов
    sessions['is_wandering'] = (sessions['hits'] >= pageview_threshold)
    return sessions[sessions['is_wandering']].sort_values('hits', ascending=False)


In [1]:
det = detect_wandering(jo)
det

NameError: name 'detect_wandering' is not defined

In [ ]:
def count_backtracks(joined_df):
    # grouped by session, sorted by hit time
    def backtracks_for_session(df):
        urls = list(df['URL'].fillna(''))
        bt = 0
        for i in range(2, len(urls)):
            if urls[i] == urls[i-2]:
                bt += 1
        return bt
    bts = joined_df.groupby('visitID').apply(lambda df: backtracks_for_session(df.sort_values('dateTime_hit')))
    return bts.rename('backtracks').reset_index()


In [74]:
count_backtracks(jo).head()

C:\Users\lirik\AppData\Local\Temp\ipykernel_7664\160873020.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bts = joined_df.groupby('visitID').apply(lambda df: backtracks_for_session(df.sort_values('dateTime_hit')))


,visitID,backtracks
0,265462931380240424,0
1,292251483796144399,2
2,373069999639363618,0
3,1255768063759351818,3
4,1436542266787758293,21


In [ ]:
def compute_funnel_conversion(joined_df, funnel_steps):
    # funnel_steps — list of URL regexes in order
    import re
    def first_step_index(urls):
        for i,pattern in enumerate(funnel_steps):
            for u in urls:
                if re.search(pattern, u):
                    return i
        return None

    sessions = joined_df.groupby('visitID').agg({'URL': lambda s: list(s)})
    sessions['first_reached'] = sessions['URL'].apply(first_step_index)
    total = len(sessions)
    per_step = []
    for i in range(len(funnel_steps)):
        reached = (sessions['first_reached'] == i).sum()
        converted_from_step = (sessions['first_reached'] >= i).sum()  # users who reached this or later
        per_step.append({'step': i, 'reached': reached, 'cumulative_reached': converted_from_step, 'percent': converted_from_step/total})
    return pd.DataFrame(per_step)
